In [19]:
def regex_to_nfa(regex):
    states = 0
    transitions = {}
    
    def new_state():
        nonlocal states
        s = states
        states += 1
        return s
    
    def add_transition(frm, sym, to):
        if frm not in transitions:
            transitions[frm] = []
        transitions[frm].append((sym, to))
    
    def single_char_nfa(c):
        s = new_state()
        e = new_state()
        add_transition(s, c, e)
        return s, e
    
    def plus_nfa(nfa):
        s, e = nfa
        add_transition(e, '', s)
        return s, e
    
    def question_nfa(nfa):
        s, e = nfa
        ns = new_state()
        ne = new_state()
        add_transition(ns, '', s)
        add_transition(e, '', ne)
        add_transition(ns, '', ne)
        return ns, ne
    
    def or_nfa(nfa1, nfa2):
        s = new_state()
        e = new_state()
        s1, e1 = nfa1
        s2, e2 = nfa2
        add_transition(s, '', s1)
        add_transition(s, '', s2)
        add_transition(e1, '', e)
        add_transition(e2, '', e)
        return s, e
    
    def concat_nfa(nfa1, nfa2):
        s1, e1 = nfa1
        s2, e2 = nfa2
        add_transition(e1, '', s2)
        return s1, e2
    
    i = 0
    def parse_atom():
        nonlocal i
        c = regex[i]
        i += 1
        nfa = single_char_nfa(c)
        if i < len(regex):
            if regex[i] == '+':
                i += 1
                nfa = plus_nfa(nfa)
            elif regex[i] == '?':
                i += 1
                nfa = question_nfa(nfa)
        return nfa
    
    def parse_expr():
        nonlocal i
        nfa = parse_concat()
        while i < len(regex) and regex[i] == '|':
            i += 1
            nfa2 = parse_concat()
            nfa = or_nfa(nfa, nfa2)
        return nfa
    
    def parse_concat():
        nonlocal i
        nfa = parse_atom()
        while i < len(regex) and regex[i] not in '|':
            nfa2 = parse_atom()
            nfa = concat_nfa(nfa, nfa2)
        return nfa
    
    nfa = parse_expr()
    return nfa[0], nfa[1], transitions

def epsilon_closure(states, transitions):
    stack = list(states)
    closure = set(states)
    while stack:
        state = stack.pop()
        for (sym, to) in transitions.get(state, []):
            if sym == '' and to not in closure:
                closure.add(to)
                stack.append(to)
    return closure

def move(states, sym, transitions):
    result = set()
    for state in states:
        for (s, to) in transitions.get(state, []):
            # Если переход по s == '.' — значит любой символ подходит
            if s == sym or s == '.':
                result.add(to)
    return epsilon_closure(result, transitions)

def nfa_to_dfa(start, accept, transitions):
    start_closure = frozenset(epsilon_closure({start}, transitions))
    dfa_states = {start_closure: 0}
    dfa_transitions = {}
    dfa_accepts = set()
    unmarked = [start_closure]
    states_count = 1
    
    while unmarked:
        current = unmarked.pop()
        dfa_transitions[dfa_states[current]] = {}
        symbols = set()
        for st in current:
            for (sym, to) in transitions.get(st, []):
                if sym != '':
                    symbols.add(sym)
        for sym in symbols:
            nxt = frozenset(move(current, sym, transitions))
            if nxt not in dfa_states:
                dfa_states[nxt] = states_count
                states_count += 1
                unmarked.append(nxt)
            dfa_transitions[dfa_states[current]][sym] = dfa_states[nxt]
    for states_set, dfa_st in dfa_states.items():
        if accept in states_set:
            dfa_accepts.add(dfa_st)
    return 0, dfa_accepts, dfa_transitions

def dfa_match(dfa_start, dfa_accepts, dfa_transitions, string):
    current = dfa_start
    for c in string:
        trans = dfa_transitions.get(current, {})
        # Если есть переход по конкретному символу — идём по нему
        if c in trans:
            current = trans[c]
        # Иначе если есть переход по '.' — идём по нему
        elif '.' in trans:
            current = trans['.']
        else:
            return False
    return current in dfa_accepts

def match_regex(regex, string):
    start, accept, transitions = regex_to_nfa(regex)
    dfa_start, dfa_accepts, dfa_transitions = nfa_to_dfa(start, accept, transitions)
    return dfa_match(dfa_start, dfa_accepts, dfa_transitions, string)

# Пример:
regex = input("Введите регулярное выражение (поддержка |, +, ?, .): ")
string = input("Введите строку для проверки: ")
print("Подходит?", match_regex(regex, string))


Введите регулярное выражение (поддержка |, +, ?, .): .h+.?
Введите строку для проверки: ehhhhhhhhhhhhhhhe
Подходит? True
